# MSiA 431 - Big Data - Homework 3 - Iteration 2 

## Kristiyan Dimitrov

In [37]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, countDistinct
from pyspark.sql import Window
from pyspark.sql.types import StructType, StructField, LongType, DoubleType
from pyspark.sql import functions as F
from pyspark.sql.functions import lit
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.sql.functions import col, weekofyear, year, month, window, count, lag, first, last, desc 
from operator import add
import pandas as pd
import numpy as np

In [11]:
spark = SparkSession.builder.appName('Problem 3').getOrCreate()

In [12]:
csv_path = 'data.csv'

In [13]:
df = spark.read.csv(csv_path, header = True , inferSchema = True, timestampFormat='YYYY-MM-DD HH:MM:SS a')

## Putting it all together in a for loop

In [14]:
# CONFIGURATION
alpha = .2

In [15]:
## This is the EWMA function, which takes a list of values and an alpha parameters and calculates the exponentially weighted average
## Testing EWMA with a range of values
def ewma_lst(alpha, lst):
    
    res = 0
    
    for ii in range(len(lst)):
        res += alpha * ( (1-alpha)**ii ) * lst[ii]
    
    return res

In [16]:
# Register UDF
ewma = spark.udf.register("ewma_lst", ewma_lst)

In [17]:
# Taking only data after 2008, before that is not relevant
df_2008 = df.filter(df.time_stamp >= '2008-01-01 00:00:00')

# Dropping 'direction' column
df_2008 = df_2008.drop('direction')

# Taking first chunk of data; NEED TO PARAMETRIZE WITH YEAR & MONTH LATER <------------------ <------------------ <------------------ <------------------ <------------------
df_1 = df_2008.filter(df_2008.time_stamp <= '2008-06-01 00:00:00')

# Let's take a subset of columns for ease
df_subset = df_1.select(['bar_num', 'profit', 'trade_id'])

In [18]:
# Verifying the max bar value across all trades
max_bar_per_trade = df_1.groupBy(col("trade_id")).agg({"bar_num": "max"}).alias('max_bar_num')
max_bar = max_bar_per_trade.agg({"max(bar_num)": "max"}).collect()[0][0]
max_bar

120

In [19]:
# Create schema for empty dataframe, which will hold all the calculated ewma profits
schema = StructType([StructField('trade_id', LongType(), False),
                     StructField('profit_ewma', DoubleType(), False), 
                     StructField('bar_num', LongType(), True)])

results = spark.createDataFrame([], schema)

In [20]:
df_subset.cache()

DataFrame[bar_num: int, profit: int, trade_id: int]

In [21]:
# The goal of this for loop is to calculate a feature based on profit

for ii in range(11, max_bar): # For bars 1, 2, 3, ... 10, we don't need to do anything; So, when we do left join, those feature values for bars 1-10 should be null
    
#     print(f'Start {ii}')
    
    if ii % 10 == 0: # This means we are in the situation of taking bar 20, 30, 40, etc.
        bars_to_take = ii - 10 # For bar 20, we want bars 10 and below; for bar 30, we want bars 20 and below...
    else:
        bars_to_take = ii - ii%10 # E.g. if we are at bar 33, we want bars 33 - 3 = 30 and below
    
    # Taking only the part of the dataset, which contains the subset of bars we are interested in
    df_filtered = df_subset.filter(f'bar_num <= {bars_to_take}')
        
    # Collecting all the profits for a given trade_id in one place (list with its corresponding trade_id)
    df_intermediate = df_filtered.groupby('trade_id').agg(F.collect_list('profit'))
        
    # Apply the UDF EWMA function to the collected list of profits
    df_ewma = df_intermediate.select('trade_id', ewma(F.lit(alpha), col("collect_list(profit)")))
        
    # Adding the bar_num we are currently at, so we can properly join later
    final_df = df_ewma.withColumn('bar_num', lit(ii))
        
    results = results.union(final_df)   
    
#     print(f'End {ii}')
    
    

I could generate the same ewma feature for all the val## features as well (I'd just leave out the little bit of if-else logic in the beginning and have bars_to_take=ii)

In [22]:
results.show(10)

+--------+-------------------+-------+
|trade_id|        profit_ewma|bar_num|
+--------+-------------------+-------+
|    9900| 14.230217728000003|     11|
|    9852|-45.088219033600005|     11|
|   10081|  42.92491776000001|     11|
|    9879| -64.16702259200002|     11|
|   10121|-16.785674240000006|     11|
|    9946|  9.038351359999998|     11|
|   10032| 15.545000038400003|     11|
|    9775| 28.437628416000003|     11|
|    9914|  93.61254041600002|     11|
|   10090|      -37.236989952|     11|
+--------+-------------------+-------+
only showing top 10 rows



In [23]:
# results.filter(results.trade_id == 9900).show() # Tried to let this run for ~ 10 minutes, but it didn't finish; aborting and hoping this works on distributed

In [24]:
# Now I need to join the original dataframe with the results i.e. to add the profit_ewma feature 

In [25]:
df_1.show()

+-------------------+-------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+
|         time_stamp|bar_num|profit|var12|var13|var14|var15|var16|var17|var18|var23|var24|var25|var26|var27|var28|var34|var35|var36|var37|var38|var45|var46|var47|var48|var56|var57|var58|var67|var68|var78|trade_id|
+-------------------+-------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+
|2008-04-10 04:19:00|    120|  -131|    9|    7|    7|    7|    7|    9|    9|    6|    6|    6|    7|    7|    7|    6|    6|    7|    9|    9|    9|    9|    9|    9|    9|    9|    9|   -1|    6|    6|    9853|
|2008-04-10 04:18:00|    119|   -97|    3|    2|    2|    2|    2|    3|    3|    1|    1|    1|    1|    2|    2|    1|    1|    2|    3|    3|

In [26]:
# df_with_new_column = df_1.join(results, on = ['trade_id', 'bar_num']) # This gave an error on my laptop which I hope won't occur on the cluster

### Assuming the join works, now I need to train a model

### The name of the new column is: profit_ewma

#### Note that this variable - profit_ewma - should be the same for all 10 bars in a given set of 10 e.g. the values for trade 9853 for bars 11-20 should be hte same, because they are all calculated based on bars 1-10.

#### Therefore, I am respecting the assignment condition that profits for bars 11-20 not be available in predicting profits for these bars (when there is the same value, then they are not influencing the prediction and only the features influence the prediction)

In [4]:
df_features = df_with_new_column.drop('time_stamp', 'bar_num', 'trade_id')

NameError: name 'df_with_new_column' is not defined

In [35]:
# Making sure profit_ewma is a float
df_features = df_features.withColumn("profit_ewma", df_features["profit_ewma"].cast(DoubleType()))

NameError: name 'df_features' is not defined

### Since all the variables are numeric I don't need Indexers & one-hot encoder 

In [31]:
vectorAssembler = VectorAssembler(inputCols = ['profit_ewma','var12','var13','var14','var15','var16','var17','var18','var23','var24','var25','var26','var27','var28','var34','var35','var36','var37','var38','var45','var46','var47','var48','var56','var57','var58','var67','var68','var78'], outputCol = 'features')

In [27]:
rf = RandomForestRegressor(featuresCol = 'features', labelCol = 'profit', seed=42)

In [32]:
pipeline = Pipeline(stages = [vectorAssembler, rf])

In [33]:
# Training model
model = pipeline.fit(df_features)

# Applying model to data (This is applying the model to the training data; for the homework I should apply to month 7 in 2008)
predictions = model.transform(df_features)

NameError: name 'df_features' is not defined

In [ ]:
# These are supposed to be the two columns needed for calculating MAPE
label_and_prediction = predictions.select('profit', 'prediction')

In [ ]:
# Need to add a small value to profit column to avoid dividing by 0
label_and_prediction = label_and_prediction.withColumn('profit', df.profit + 0.1) 

In [ ]:
MAPE = label_and_prediction.withColumn('percent_difference', expr("F.abs(profit - prediction)/profit"))

In [ ]:
avg_MAPE = MAPE.select(F.mean(col('percent_difference')).alias('mean')).collect()

mean = avg_MAPE[0]['mean']
print(mean)